# Machine Learning Prediction in Real Time Using Docker and Python REST APIs with Flask

In this project, I want to show how to deploy a machine learning algorithm with a Python-Flask RESTful-API around a localhost Docker container and get real-time online predictions.

To learn this concept, I will implement online inferences (Linear Discriminant Analysis and Multi-layer Perceptron Neural Network models) with Docker and Flask-RESTful.

I found a pre-trained engine on EEG data that predicts the alphabet letter the human subject had thought.

This guide wrote for Windows Terminal and if you have another OS you should change it if you need.

In this guide I got some codes from @xaviervasques. 

## Install requirements

Before we start, we should [install Docker](https://docs.docker.com/desktop/) on our computer. If you have Windows OS, it would be better to activate and install [Windows Subsystem for Linux (WSL) Version 2](https://docs.microsoft.com/en-us/windows/wsl/install) and then ubuntu distribution on it.

After installation of Docker, we need to install [Python](https://hub.docker.com/_/python), [Scipy-notebook](https://hub.docker.com/r/jupyter/scipy-notebook) and [curl](https://hub.docker.com/r/curlimages/curl) on Docker.

In [ ]:
docker pull python
docker pull jupyter/scipy-notebook
docker pull curlimages/curl

The next step is to clone [ML online prediction repository](https://github.com/magnooj/ml-online-prediction) from GitHub.

In [ ]:
git clone https://github.com/magnooj/ml-online-prediction.git
cd ml-online-prediction

## Through the files

Now, we are ready to run and test our project. By running `ls` you can see these files:

- [`api.py`](https://github.com/magnooj/ml-online-prediction/blob/main/api.py) : Python-Flask RESTful-API
- [`Dockerfile`](https://github.com/magnooj/ml-online-prediction/blob/main/Dockerfile) : Docker container
- [`README.md`](https://github.com/magnooj/ml-online-prediction/blob/main/README.md) : Instructions
- [`requirements.txt`](https://github.com/magnooj/ml-online-prediction/blob/main/requirements.txt) : Required modules to run this project
- [`test.json`](https://github.com/magnooj/ml-online-prediction/blob/main/test.json) : Dataset for test the app
- [`train.csv`](https://github.com/magnooj/ml-online-prediction/blob/main/train.csv) : Dataset for train the app
- [`train.py`](https://github.com/magnooj/ml-online-prediction/blob/main/train.py) : Machine Learning app

The ``train.py`` is a python script that ingest and normalize EEG data and train two models to classify the data. The `Dockerfile` will be used to build our Docker image, `requirements.txt` *(flask, flask-restful, joblib)* is for the Python dependencies and `api.py` is the script that will be called to perform the online inference using *REST APIs*. `train.csv` are the data used to train our models, and `test.json` is a file containing new EEG data that will be used with our inference models.

### ***Flask RESTful APIs***

The first step in building *APIs* is to think about the data we want to handle, how we want to handle it and what output we want with our *APIs*. In our example, we will use the `test.json` file in which we have 1300 rows of EEG data with 160 features each (columns). We want our *APIs* to the following:

- ***API 1***: We will give a row number to the API which will extract for us the data from the selected row and print it.
- ***API 2***: We will give a row number to the API which will extract the selected row, inject the new data into the models and retrieve the classification prediction (Letter variable in the data).
- ***API 3***: We will ask the API to take all the data in the test.json file and instantly print us the classification score of the models.

At the end, we want to access those processes by making an HTTP request.
Let’s have a look at the `api.py` file:

In [ ]:
import http
import json
import os
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
import pandas as pd
from joblib import load
from sklearn import preprocessing

from flask import Flask

# Set environment variables
MODEL_DIR = os.environ["MODEL_DIR"]
MODEL_FILE_LDA = os.environ["MODEL_FILE_LDA"]
MODEL_FILE_NN = os.environ["MODEL_FILE_NN"]
MODEL_PATH_LDA = os.path.join(MODEL_DIR, MODEL_FILE_LDA)
MODEL_PATH_NN = os.path.join(MODEL_DIR, MODEL_FILE_NN)

# Loading LDA model
print("Loading model from: {}".format(MODEL_PATH_LDA))
inference_lda = load(MODEL_PATH_LDA)

# loading Neural Network model
print("Loading model from: {}".format(MODEL_PATH_NN))
inference_NN = load(MODEL_PATH_NN)

# Creation of the Flask app
app = Flask(__name__)

# API 1
# Flask route so that we can serve HTTP traffic on that route
@app.route('/line/<Line>')
# Get data from json and return the requested row defined by the variable Line
def line(Line):
    with open('./test.json', 'r') as jsonfile:
       file_data = json.loads(jsonfile.read())
    # We can then find the data for the requested row and send it back as json
    return json.dumps(file_data[Line])
    

# API 2
# Flask route so that we can serve HTTP traffic on that route
@app.route('/prediction/<int:Line>',methods=['POST', 'GET'])
# Return prediction for both Neural Network and LDA inference model with the requested row as input
def prediction(Line):
    data = pd.read_json('./test.json')
    data_test = data.transpose()
    X = data_test.drop(data_test.loc[:, 'Line':'# Letter'].columns, axis = 1)
    X_test = X.iloc[Line,:].values.reshape(1, -1)
    
    clf_lda = load(MODEL_PATH_LDA)
    prediction_lda = clf_lda.predict(X_test)
    
    clf_nn = load(MODEL_PATH_NN)
    prediction_nn = clf_nn.predict(X_test)
    
    return {'prediction LDA': int(prediction_lda), 'prediction Neural Network': int(prediction_nn)}

# API 3
# Flask route so that we can serve HTTP traffic on that route
@app.route('/score',methods=['POST', 'GET'])
# Return classification score for both Neural Network and LDA inference model from the all dataset provided
def score():

    data = pd.read_json('./test.json')
    data_test = data.transpose()
    y_test = data_test['# Letter'].values
    X_test = data_test.drop(data_test.loc[:, 'Line':'# Letter'].columns, axis = 1)
    
    clf_lda = load(MODEL_PATH_LDA)
    score_lda = clf_lda.score(X_test, y_test)
    
    clf_nn = load(MODEL_PATH_NN)
    score_nn = clf_nn.score(X_test, y_test)
    
    return {'Score LDA': score_lda, 'Score Neural Network': score_nn}

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0')


The first step, after importing dependencies including the open source web micro-framework Flask, is to set the environment variables that are written in the Dockerfile. We also need to load our *Linear Discriminant Analysis (LDA)* and *Multi-layer Perceptron Neural Network (NN)* serialized models. We create our Flask application by writing `app = Flask(__name__)`. Then, we create our three Flask routes so that we can serve HTTP traffic on that route:

- http://localhost:5000/line/250 : Get data from `test.json` and return the requested row defined by the variable Line (in this example we want to extract the data of row number 250).
- http://localhost:5000/prediction/51 : Returns classification prediction from both LDA and Neural Network trained models by injecting the requested data (in this example, we want to inject the data of row number 51).
- http://localhost:5000/score : Return classification score for both the *Neural Network* and *LDA* inference models on all the available data (`test.json`).

The Flask routes allows us to request what we need from the API by adding the name of our procedure (`/line/<Line>`, `/prediction/<int:Line>`, `/score`) to the URL (http://localhost:5000). Whatever the data we add, `api.py` will always return the output we request.

### Machine Learning models
The `train.py` is a python script that ingests and normalizes EEG data in a csv file (`train.csv`) and train two models to classify the data (using `scikit-learn`). The script saves two models: Linear Discriminant Analysis (`clf_lda`) and Neural Networks multi-layer perceptron (`clf_NN`):

In [ ]:
#!/usr/bin/python
# tain.py
# Debug and edit: Ali Ganjizadeh(@magnooj) 22/01/2022; Original code: @xaviervasques

import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)

import os
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
import pandas as pd
from joblib import dump
from sklearn import preprocessing

def train():

    # Load directory paths for persisting model
    MODEL_DIR = os.environ["MODEL_DIR"]
    MODEL_FILE_LDA = os.environ["MODEL_FILE_LDA"]
    MODEL_FILE_NN = os.environ["MODEL_FILE_NN"]
    MODEL_PATH_LDA = os.path.join(MODEL_DIR, MODEL_FILE_LDA)
    MODEL_PATH_NN = os.path.join(MODEL_DIR, MODEL_FILE_NN)
      
    # Load, read and normalize training data
    training = "./train.csv"
    data_train = pd.read_csv(training)
        
    y_train = data_train['# Letter'].values
    X_train = data_train.drop(data_train.loc[:, 'Line':'# Letter'].columns, axis = 1)

    print("Shape of the training data")
    print(X_train.shape)
    print(y_train.shape)
        
    # Data normalization (0,1)
    X_train = preprocessing.normalize(X_train, norm='l2')
    
    # Models training
    
    # Linear Discrimant Analysis (Default parameters)
    clf_lda = LinearDiscriminantAnalysis()
    clf_lda.fit(X_train, y_train)
    
    # Serialize model
    from joblib import dump
    dump(clf_lda, MODEL_PATH_LDA)
        
    # Neural Networks multi-layer perceptron (MLP) algorithm
    clf_NN = MLPClassifier(solver='adam', activation='relu', alpha=0.0001, hidden_layer_sizes=(500,), random_state=0, max_iter=1000)
    clf_NN.fit(X_train, y_train)
       
    # Serialize model
    from joblib import dump, load
    dump(clf_NN, MODEL_PATH_NN)
        
if __name__ == '__main__':
    train()


## Run APIs through a Docker container

We have all to build our [Docker Image](https://docs.docker.com/engine/reference/commandline/images/). To start, we need our `Dockerfile` with the `jupyter/scipy-notebook` image as our base image. We also need to set our environment variables and install `joblib` to allow serialization and deserialization of our trained models and flask (`requirements.txt`).

### ***Create a Docker Image***

After installation of Docker App, we should create a [Docker Image](https://docs.docker.com/engine/reference/commandline/images/). Thus, we copy the `train.csv`, `test.json`, `train.py` and `api.py` files into the image. Then, we run `train.py` which will fit and serialize the machine learning models as part of our image build process.

Here is the Dockerfile code:

In [ ]:
FROM jupyter/scipy-notebook

RUN mkdir my-model
ENV MODEL_DIR=/home/jovyan/my-model
ENV MODEL_FILE_LDA=clf_lda.joblib
ENV MODEL_FILE_NN=clf_nn.joblib

COPY requirements.txt ./requirements.txt
RUN pip install -r requirements.txt 

COPY train.csv ./train.csv
COPY test.json ./test.json

COPY train.py ./train.py
COPY api.py ./api.py


RUN python train.py


To build this image, run the following command:

In [ ]:
docker build -t my-docker-api -f Dockerfile .

We will get this output:

<img src="https://github.com/magnooj/ml-online-prediction/blob/main/images/1.png?raw=true" alt="Picture of copying files in container">

We can see our images by `Docker images` command: :

<img src="https://github.com/magnooj/ml-online-prediction/blob/main/images/2.png?raw=true" alt="Picture of Docker repositories">

### ***Serve a Docker container***

Now the goal is to run our online inference meaning that each time a client issues a POST request to the `/line/<Line>`, `/prediction/<Line>`, `/score endpoints`, we will show the requested data (row), predict the class of the data we inject using our pre-trained models, and the score of our pre-trained models using all the available data. To launch the web server, we will run a Docker container and run the `api.py` script:

In [ ]:
docker run -it -p 5000:5000 my-docker-api python api.py

The `-p` flag exposes port `5000` in the container to `port 5000` on our host machine, `-it` flag allows us to see the logs from the container and we run `python api.py` in the `my-api` image.

The output is the following:

<img src="https://github.com/magnooj/ml-online-prediction/blob/main/images/3.png?raw=true" alt="Picture of Docker server start">

You can see that we are running on http://localhost:5000/ and we can now use our web browser or the `curl` command to issue a POST request to the IP address.

If we type:

In [ ]:
curl http://localhost:5000/line/232

We will get row number 232 extracted from our data (`test.json`):

<img src="https://github.com/magnooj/ml-online-prediction/blob/main/images/4.png?raw=true" alt="Picture curl jason data">

Same result using the web browser:

<img src="https://github.com/magnooj/ml-online-prediction/blob/main/images/5.png?raw=true" alt="Picture of browser data">

If we type the following `curl` command:

In [ ]:
curl http://localhost:5000/prediction/232

We will see the following output:

<img src="https://github.com/magnooj/ml-online-prediction/blob/main/images/6.png?raw=true" alt="Picture of predicting the given EEG">

The above output means that the LDA model classified the provided data (*row 232*) as letter 21 (*U*) while Multi-layer Perceptron Neural Network classified the data as letter 8 (*H*). The two models do not agree.

If we type:

In [ ]:
curl http://lohalhost:5000/score

We will see the score of our models on the entire dataset:

<img src="https://github.com/magnooj/ml-online-prediction/blob/main/images/7.png?raw=true" alt="Picture of accuracy of two models">

As we can read, we should trust more the Multi-layer Perceptron Neural Network with an accuracy score of `0.59` even if the score is not so high.

I hope you enjoyed how to containerizing your machine/deep learning applications using Docker and flask to perform online inference. if you have any comments please do not hesitate to send me an [e-mail](mailto:magnooj@gmail.com).

Regards,

Ali Ganjizadeh
